# QPSK Benchmarking

This notebook benchmarks the BER/SNR, BER/CFO, and BER/STO of a QPSK communication chain.

## Algorithm
1. Generate transmit signal
2. Define reception function
3. Individually scan over SNR, CFO, and STO, measuring BER
4. Plot results

Author: Cole Delong

### Imports and Constants

In [6]:
%reload_ext autoreload
%autoreload 2

import py_utils as pu
import py_utils.visualization as vz
import numpy as np

N_BITS = 10**5

### Transmit chain

In [2]:
bits_tx = np.random.randint(2, size=N_BITS)
sym_tx = pu.coding.diff_encode_psk_symbols(pu.modulation.modulate_qpsk(bits_tx))


SPS = 2
N_RRC_TAPS = SPS*10 + 1
sig_tx = pu.dsp.upsample(sym_tx, SPS)
sig_tx = pu.dsp.rrc_filter(sig_tx, N_RRC_TAPS, SPS)

### Recieve chain function

In [3]:
def receive_chain(sig_rx, sps=SPS, n_rrc_taps=N_RRC_TAPS):
    # Matched filter
    sig_matched = pu.dsp.rrc_filter(sig_rx, N_RRC_TAPS, SPS)

    # Timing recovery
    stc = pu.timing_recovery.GardnerSymbolTimingCorrector()
    symbols_sampled = stc.process(sig_matched)

    # Carrier recovery
    loop_bw = 1/20
    costas = pu.carrier_recovery.CostasLoopQPSK(loop_bw)
    sym_rot = np.empty_like(symbols_sampled, dtype=np.complex64)
    costas.process(symbols_sampled, sym_rot)

    # Demodulate
    bits_rx = pu.modulation.demodulate_qpsk(pu.coding.diff_decode_psk_symbols(sym_rot))

    return bits_rx

### Scan over SNR

In [ ]:
# BER vs SNR with CFO and STO

snr_db_range = range(0, 31, 5)
ber_values = []

# Pass through channel with CFO and STO
CFO_PCT = 0.03  # 3% CFO
STO = 0.3  # fractional symbol offset
sig_chan = pu.channel.apply_cfo(sig_tx, CFO_PCT)
sig_chan = pu.channel.apply_sto(sig_chan, STO)

# Scan over SNR values
for SNR_DB in snr_db_range:
    sig_chan_awgn = pu.channel.apply_awgn(sig_chan, SNR_DB)

    bits_rx = receive_chain(sig_chan_awgn)
    ber = np.sum(bits_tx != bits_rx) / N_BITS
    ber_values.append(ber)

# Plot results
_ = vz.plot_signal(ber_values, x=snr_db_range,
    title="QPSK BER vs SNR with CFO and STO",
    xlabel="SNR (dB)",
    ylabel="Bit Error Rate (BER, DB scale)",
    db=True,)




### Scan over CFO

### Scan over STO